# FIT5196 Assessment 3
#### Student Names and Student Id : Akshay Rai Chopra(30228751) and  Parul (29507960)
#### Group Number: 154


Date: 04/10/2019

Version: 1.0

Environment: Python 2.7.11 and Jupyter notebook

## Introduction:
### Data Cleansing
Data cleansing is an iterative process. The first step of the cleansing process is data auditing. In this step, we identify the types of anomalies that reduce the data quality.  Data auditing is about programmatically checking the data using some validation rules that are pre-specified, and then creating a report of the quality of the data and its problems. We often apply some statistical tests in this step for examining the data.
Data Anomalies can be classified at a high level into three categories:

1. **Syntactic Anomalies**: 
describe characteristics concerning the format and values used for representation of the entities. Syntactic anomalies such as: lexical errors, domain format errors, syntactical error and irregularities.
2. **Semantic Anomalies**: 
hinder the data collection from being a comprehensive and non-redundant representation of the mini-world. These types of anomalies include: Integrity constraint violations, contradictions, duplicates and invalid tuples
3. **Coverage Anomalies**: 
decrease the amount of entities and entity properties from the mini-world that are represented in the data collection. Coverage anomalies are categorized as: missing values and missing tuples


"Group154_dirty_data.csv" - The dirty data which have different type of errors and we need to detect and fix them.

"Group154_outlier_data.csv" - The data which have outlier w.r.t delivery fee column, we need to remove the rows that have outliers

"Group154_missing_data.csv" - The data which have missing values and we need to impute them correctly.

The dataset contains Food Delivery data from a restaurant in Melbourne, Australia. The restaurant has three branches around CBD area. All three branches share the same menu but they have different management so they operate differently.

 

### Import libraries
For the assignment we use a number of libraries to help us process and fix the data.

In [1]:
import pandas as pd
import re
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

<b> As a first step we load the data using Pandas library and store them in the respective dataframes by "pd.read_csv()"

In [2]:
dirty_data = pd.read_csv("Group154_dirty_data.csv")
outlier_data = pd.read_csv("Group154_outlier_data.csv")
missing_data = pd.read_csv("Group154_missing_data.csv")


### Firstly we will fix the errors in Dirty_data

In [3]:
print(dirty_data.shape) # .shape tells the number of rows and columns in the dateframe

(500, 12)


We have 500 rows(observations) and 15 columns(variables)

In [4]:
dirty_data.info() # .info() gives an insight about which features are categorical and which are numerical.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
order_id                   500 non-null object
date                       500 non-null object
time                       500 non-null object
order_type                 500 non-null object
branch_code                500 non-null object
order_items                500 non-null object
order_price                500 non-null float64
customer_lat               500 non-null float64
customer_lon               500 non-null float64
customerHasloyalty?        500 non-null int64
distance_to_customer_KM    500 non-null float64
delivery_fee               500 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


In [5]:
dirty_data.describe() # .describe() tells the distribution(statisitcs) of numerical values.

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,511.842300,-30.753763,143.503721,0.120000,8.739410,13.813120
std,266.651019,25.336331,16.298857,0.325287,1.600082,2.539826
min,45.250000,-37.828216,-37.819026,0.000000,3.271000,4.499288
25%,302.812500,-37.817794,144.950582,0.000000,7.813250,12.631459
50%,465.350000,-37.811639,144.963488,0.000000,8.792000,14.006261
75%,683.500000,-37.804107,144.978716,0.000000,9.848750,15.300665
max,1520.000000,144.976883,145.016633,1.000000,13.043000,21.119195


The observation tells us that:

* Total samples are 500
* customerHasloyalty?  is a categorical feature with 0 or 1 values.
* Max of customer_lat is 144 and which is not in the range of latitudes
* Min of customer_lon is -37 and which is not in the range of longitudes
* order price has a maximum value of 1520, it is strange ( possibly an error )

In [6]:
dirty_data.describe(include=['O']) #distribution of categorical values

,order_id,date,time,order_type,branch_code,order_items
count,500,500,500,500,500,500
unique,500,303,72,3,6,497
top,ORDB08554,2018-10-31,18:08:27,Lunch,TP,"[('Salad', 6), ('Chicken', 10)]"
freq,1,5,12,170,163,2


the table above tells us:

* branch_code variable has 6 possible values but we are provided with the info that there are only 3 branches.
* Order_type has only 3 values, lunch, breakfast and dinner with count of Lunch being 170 times



### Fixing the branch code

In [7]:
dirty_data.branch_code.value_counts() #.value_counts return a series containing counts of unique values.

TP    163
BK    155
NS    149
ns     14
tp     10
bk      9
Name: branch_code, dtype: int64

There is a lexical error (syntactic anomaly) - branch_code can only take 3 values since there are only 3 branches. We can clearly infer that ns means NS, tp means TP and bk means BK. So we will fix this error but before that we try to see if the branch_code has any relation wtih order_id, so we use the data present in outlier to see any kind of relation since there zre no errors in the outlier_data.


In [8]:
# creating 3 emoty lists
ns_o_list=[]
tp_o_list=[]
bk_o_list=[]


#iterrows() function will loop through each row of a dataframe as (index, series) pairs
#for loop along with iterrows will help to iterate thrrough each row one by one.
for a,b in outlier_data.iterrows(): 
    b_code=b['branch_code'] # saving each branch_code in b_code
    o_id=b['order_id']      # saving each order_id in o_id 
    
    
    #if the branch code is NS, appending the first 4 characters of order_id in ns_o_list 
    if b_code =='NS':
        ns_o_list.append(o_id[0:4])
        
    #if the branch code is TP, appending the first 4 characters of order_id in tp_o_list    
    if b_code =='TP':
        tp_o_list.append(o_id[0:4])
        
    #if the branch code is BK, appending the first 4 characters of order_id in bk_o_list
    if b_code =='BK':
        bk_o_list.append(o_id[0:4])
    
#using set on the list will give the distinct values of corresponding order_ids
print('ns_o_list - ', set(ns_o_list)) 
print('tp_o_list - ', set(tp_o_list))
print('bk_o_list - ', set(bk_o_list))

ns_o_list -  {'ORDZ', 'ORDC', 'ORDI'}
tp_o_list -  {'ORDY', 'ORDB', 'ORDJ'}
bk_o_list -  {'ORDA', 'ORDX', 'ORDK'}


Yes, there is a relation between order_id and branch_code and we will see if there are any errors regarding this. 

In [9]:
dirty_data[['order_id','branch_code']].head(20)

,order_id,branch_code
0,ORDJ07617,TP
1,ORDY05853,TP
2,ORDI02968,NS
3,ORDY06446,TP
4,ORDZ01045,NS
5,ORDA06887,BK
6,ORDB08630,TP
7,ORDZ09998,NS
8,ORDY01587,TP
9,ORDZ05882,tp


See the 9th and 19th row , ORDZ05882 should belong to NS but is assigned to tp, whereas ORDX09527 should belong to BK but is assigned to tp. Another thing , correcting these errors might fix the lexical error we found above.

In [10]:

#iterrows() function will loop through each row of a dataframe as (index, series) pairs
#for loop along with iterrows will help to iterate thrrough each row one by one.

for x,y in dirty_data.iterrows():
    o_id = y['order_id'] # saving each order_id in o_id
    b_code = y['branch_code'] # saving each branch_code in b_code
    
    
    # if the first 4 characters of order_id are in the ns_o_list we created above, then 
    # we check another if condition that if b_code is not equal to 'NS
    # then assign the value 'NS' to it 
    # iloc[x,4] means the datavalue with x as the index of the row and 4 means the column 'branch_code'
    if o_id[0:4] in ns_o_list:
        if b_code!= 'NS':
            dirty_data.iloc[x,4] = 'NS'
            
    # if the first 4 characters of order_id are in the tp_o_list we created above, then 
    # we check another if condition that if b_code is not equal to 'TP'
    # then assign the value 'TP' to it 
    # iloc[x,4] means the datavalue with x as the index of the row and 4 means the column 'branch_code'        
    elif o_id[0:4] in tp_o_list:
        if b_code!= 'TP':
            dirty_data.iloc[x,4] = 'TP'
    
    # if the first 4 characters of order_id are in the bk_o_list we created above, then 
    # we check another if condition that if b_code is not equal to 'BK'
    # then assign the value 'BK' to it 
    # iloc[x,4] means the datavalue with x as the index of the row and 4 means the column 'branch_code'
    elif o_id[0:4] in bk_o_list:
        if b_code!= 'BK':
            dirty_data.iloc[x,4] = 'BK'

In [11]:
dirty_data.branch_code.value_counts() 

TP    172
BK    165
NS    163
Name: branch_code, dtype: int64

Both the errors in the brancg_code are now fixed.

### Fixing the format of Date

In [12]:
dirty_data.date.head(10) #seeing the first 10 values of column 'date' using .head(10) 

0    2018-08-17
1    2018-06-26
2    2018-01-26
3    2018-06-24
4    2018-10-02
5    11-03-2018
6    2018-11-26
7    2018-07-16
8    2018-03-26
9    2018-11-14
Name: date, dtype: object

The format of the date is not consistent. Most of the values have yyyy-mm-dd format and there are some with mm-dd-yyyy or dd-mm-yyyy format which needs to be fixed. This is another syntactic error.

We first find out the dates which are in correct format or incoorect format with the help pf regular expression

In [13]:
reg_date1= re.compile(r"(\d{4})-(\d{2})-(\d{2})") #it matches the date in yyyy-mm-dd or yyyy-dd-mm format
reg_date2= re.compile(r"(\d{2})-(\d{2})-(\d{4})") #it matches the date in dd-mm-yyyy format

#performing iterations on every row to check the date format 
for n,j in dirty_data.iterrows():
    hold_date=j['date']                            #storinf the date for each row into variable named hold_date
    date1=re.findall(reg_date1,hold_date)          #performing regex using reg_date1 to extract the date of format yyyy-mm-dd or yyyy-dd-mm
    if len(date1) == 0:                            #if nothing extracted, that means the date is of the format dd-mm-yyyy
        date2=re.findall(reg_date2,hold_date)[0]   #performing another regex to extract the list of data of three groups (day,month,year) induvidually
        #ectracted data is used to reverse the order and print date in correct format using string concatination
        dirty_data.iloc[n,1] = date2[2]+'-'+date2[1]+'-'+date2[0]
        
    else:                                          #else something is extracted of the form yyyy-mm-dd or yyyy-dd-mm
        #if the value of the second group is greater than 12, then, its the date as the month count can never be greater than 12
        if int(date1[0][1])>12:
            #interchange the 2nd and 3rd group value for resulting format to be yyyy-mm-dd
            dirty_data.iloc[n,1] = date1[0][0]+'-'+date1[0][2]+'-'+date1[0][1]
    #otherwise the date is already in the correct format tthat is yyyy-mm-dd.
            
#the dates have been fixed now.

### Fixing the correct order_type with respect to the time given.

In [14]:
dirty_data[['time','order_type']].tail(14) #seeing the last 14 values of columns 'time' and 'order_type' using .tail(14) 

,time,order_type
486,14:35:29,Breakfast
487,17:58:18,Dinner
488,11:43:05,Breakfast
489,08:50:42,Breakfast
490,17:58:18,Dinner
491,16:57:27,Dinner
492,15:26:11,Breakfast
493,16:16:54,Dinner
494,10:21:58,Breakfast
495,16:57:27,Dinner



We are provide with the following information 

There are three types of meals:
* Breakfast - served during morning (8am - 12pm),
* Lunch - served during afternoon (12:00:01pm - 4pm)
* Dinner - served during evening (4:00:01pm - 8pm)

Check the row 486 and 492. The time of the order doesn't correspond to the order type. The order_type is breakfast instead of lunch. Therefore the order_type needs to be fixed.

In [15]:
#iterrows() function will loop through each row of a dataframe

for n,j in dirty_data.iterrows():
    time = j['time']        #saving each value of'time' in time
    order = j['order_type'] #saving each value of 'order_type' in order
    
    
    #if the time is in between 8am to 12pm and order_type is not equal to Breakfast then use iloc to loacte the error and fix it by assigning 'Breakfast' to it.
    #.iloc[n,3] the datavalue with n as the index of the row and 3 means the column 'order_type'
    if (time >= '08:00:00') & (time <= '12:00:00') & (order != 'Breakfast'):
        dirty_data.iloc[n,3] = 'Breakfast' 
        
    
    #if the time is in between 12:01pm to 4pm and order_type is not equal to Lunch then use iloc to loacte the error and fix it by assigning 'Lunch' to it.
    if (time > '12:00:00') & (time <= '16:00:00') & (order != 'Lunch'):
        dirty_data.iloc[n,3] = 'Lunch'
        
        
    #if the time is in between 4:01pm to 8pm and order_type is not equal to Dinner then use iloc to loacte the error and fix it by assigning 'Dinner' to it.
    if (time > '16:00:00') & (time <= '20:00:00') & (order != 'Dinner'):
        dirty_data.iloc[n,3] = 'Dinner'


### Fixing the errors in customer_lat and cutomer_lon

In [16]:
dirty_data.customer_lat.describe() #.describe() tells the distribution(statisitcs) of customer_lat

count    500.000000
mean     -30.753763
std       25.336331
min      -37.828216
25%      -37.817794
50%      -37.811639
75%      -37.804107
max      144.976883
Name: customer_lat, dtype: float64

In [17]:
dirty_data.customer_lon.describe() #.describe() tells the distribution(statisitcs) of customer_lon

count    500.000000
mean     143.503721
std       16.298857
min      -37.819026
25%      144.950582
50%      144.963488
75%      144.978716
max      145.016633
Name: customer_lon, dtype: float64

The latitude angles range from -90$^\circ$ to 90$^\circ$

The longitude angles range from 0$^\circ$ to 180$^\circ$

On visualizing the descriptive stasitics of customer_lat of the data given, we can observe that maximum value is 144.97 which is definately wrong entry as it should be in range from -90 to 90. Similalry, the minimum value of descriptive statisitcs of customer_lon is -37.81 which is a similar error as above, out of range. 

Now we check the rows where customer_lat is not in the range(-90 to 90) 

In [18]:
dirty_data[(dirty_data['customer_lat'] > 90) | (dirty_data['customer_lat'] < -90)]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
208,ORDC07297,2018-05-24,12:43:56,Lunch,NS,"[('Chicken', 4), ('Salad', 9), ('Burger', 4), ...",760.8,144.976883,-37.819026,0,8.462,13.607341
291,ORDC00656,2018-08-06,15:05:54,Lunch,NS,"[('Burger', 2), ('Fries', 6)]",134.0,144.965627,-37.812000,0,7.662,13.471045
313,ORDZ03764,2018-07-11,14:15:12,Lunch,NS,"[('Chicken', 3), ('Burger', 10), ('Steak', 10)...",993.2,144.942916,-37.805785,0,9.165,14.381864
454,ORDY04585,2018-08-23,15:46:28,Lunch,TP,"[('Chicken', 7), ('Burger', 6), ('Fries', 1), ...",737.0,144.937511,-37.806875,0,10.488,13.212223


There are 4 rows where customer_lat is out of range and infact we can also observe that the values in customer_lat and customer_lot are swapped. In the next step we are going to rectify these 4 rows by swapping their respective values from customer_lat to customer_lon

In [19]:
#iterrows() function will loop through each row of a dataframe

for a,b in dirty_data.iterrows():
    lat = b['customer_lat'] #saving each value of'customer_lat' in lat
            
    #if the lat is greater than 90 i.e out of range
    # we swap the values of customer_lat and customer_long
    if lat >= 90:
        temp = dirty_data.iloc[a,7]  #storing the value of customer_lat in temp
        dirty_data.iloc[a,7]= dirty_data.iloc[a,8] #assigning the value of customer_lon to customer_lat 
        dirty_data.iloc[a,8]= temp #assigning the value of temp(customer_lat) to customer_lon
    
        
    


The above errors are fixed. We are provided with the nodes_data which provides the information regarding the node w.r.t latitude and longitude. So just to keep a check, we observed that latitude doesn't have any values greater than 0

In [20]:
nodes_data = pd.read_csv("nodes.csv")
nodes_data[nodes_data['lat'] > 0] # there are 0 values of lat with any postive value.

,node,lat,lon


In [21]:
dirty_data[dirty_data['customer_lat'] > 0].count()  # 37 values of latitude are positve which needs to fixed.

order_id                   37
date                       37
time                       37
order_type                 37
branch_code                37
order_items                37
order_price                37
customer_lat               37
customer_lon               37
customerHasloyalty?        37
distance_to_customer_KM    37
delivery_fee               37
dtype: int64

In [22]:
#iterrows() function will loop through each row of a dataframe
for a,b in dirty_data.iterrows():
    lat = b['customer_lat'] #saving each value of'customer_lat' in lat
    
    #if the lat is greater than 0 then multiplying it by -1 to make it negative.
    if (lat >=0):
        dirty_data.iloc[a,7] = dirty_data.iloc[a,7] * (-1)

### Fixing the errors in distance_to_customer_KM.

In [23]:
#we load the data using Pandas library and store them in the respective dataframes by "pd.read_csv()"
#these datastes provide the correct information about the locations.
 
nodes_data = pd.read_csv("nodes.csv") 
edges_data = pd.read_csv("edges.csv")
branches_data = pd.read_csv("branches.csv")

We first need to merge the dataframes and we do it by using merge. "pd.merge" merges two DataFrames or named Series objects with a database-style join. We first perform a left join on dirty_data and nodes_data so that to corresponding customer lat and lon we get the correct customer nodes


In [24]:
#'how' - left outer join on dirty_data and nodes_data 
# right_on -Columns to join on in the right DataFrame. 
# left_on - Columns to join on in the left DataFrame. 

dirty_data = pd.merge(left=dirty_data,right=nodes_data, how='left', left_on=['customer_lat','customer_lon'], right_on=['lat','lon'])
#we are merging on the same dataframe

dirty_data=dirty_data.drop(columns=['lat','lon'])
dirty_data=dirty_data.rename(columns={'node':'customer_node'})

#after joing, dropping the columns lat and lon and renaming the column 'node' to 'customer_node' 

We now perform left join on branches_data and nodes_data so that to corresponding branch lat and lon we get the correct branch nodes

In [25]:
#'how' - left outer join on dirty_data and nodes_data 
# right_on -Columns to join on in the right DataFrame. 
# left_on - Columns to join on in the left DataFrame

branches1= pd.merge(left=branches_data,right=nodes_data, how='left', left_on=['branch_lat','branch_lon'], right_on=['lat','lon'])
#creating a new datframe branches1 and then merging.

branches1=branches1.drop(columns=['lon','lat','branch_lat','branch_lon','branch_name'])
branches1=branches1.rename(columns={'node':'branch_node'})


#after joing, dropping the columns lon, lat, branch_lat, branch_lon and branch_name and renaming the column 'node' to 'branch_node'
branches1.head()

,branch_code,branch_node
0,NS,2455254505
1,TP,1390575046
2,BK,1889485053


We now need to perform left join on branches1 and dirty_data so that to to corresponding branch code we get correct branch node.

In [26]:
#'how' - left outer join on dirty_data and nodes_data 
# right_on -Columns to join on in the right DataFrame. 
# left_on - Columns to join on in the left DataFrame

dirty_data=pd.merge(left=dirty_data,right=branches1, how='left', left_on=['branch_code'], right_on=['branch_code'])
dirty_data.head(2)


,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,customer_node,branch_node
0,ORDJ07617,2018-08-17,15:36:20,Lunch,TP,"[('Chicken', 6), ('Steak', 5), ('Burger', 9), ...",730.4,-37.818790,144.955280,0,8.299,11.938508,2247180690,1390575046
1,ORDY05853,2018-06-26,13:44:47,Lunch,TP,"[('Eggs', 8), ('Chicken', 9), ('Burger', 4), (...",656.2,-37.824714,144.984714,0,8.921,12.535316,746912608,1390575046


We have branch code and customer node and now we can find out the shortest distance in km with the help of <b>Dijkstra Algorithim</b> 

In [27]:
import networkx
G = networkx.Graph() #Creating an empty graph with no nodes and no edges.
n=nodes_data.node    # storing nodes in n from the nodes_data
G.add_nodes_from(n)  # Adding the nodes 'n' to G
print('The number of nodes in the graph are ',G.number_of_nodes())

The number of nodes in the graph are  17117


In [28]:

u=edges_data['u']                  #storing 'u' in u from the edges_data
v=edges_data['v']                  #storing 'v' in v from the edges_data
distance=edges_data['distance(m)'] #storing 'distacne(m)' in distance from the edges_data

temp_list=list(zip(u,v,distance)) #zipping all the above variables and storing them in list. 

G.add_weighted_edges_from(temp_list) # addding the edges to G from the temp_list
print('The number of edges in the graph are ',G.number_of_edges())

The number of edges in the graph are  25491


In [29]:
initial_pt=dirty_data['branch_node']  #storing each 'branch_code' in initial_pt from the dirty_data
final_pt=dirty_data['customer_node']  #storing each 'customer_code' in final_pt from the dirty_data

temp_list1=list(zip(initial_pt,final_pt)) #zipping the variables and storing them in list
dis=[] #empty list

for a,b in temp_list1:
    dis.append(networkx.dijkstra_path_length(G,a,b))
    #dijkstra_path_length returns the shortest path length from source to target in a weighted graph.
    #appending all the distances to the list dis

In [30]:
#The distance we calculated is in metres and we need to convert it into km since the distance_to_customer_KM is given in kms.
#we divide each item of the list by 1000 using for loop and appending the new items into new list 'correct_dis'
correct_dis=[] 
for x in dis:
    correct_dis.append(x/1000)

In [31]:
#adding another column 'correct_dis_km' to the dataframe which have the values we found above.
dirty_data['correct_dis_km']=correct_dis

# iterrows() function will loop through each row of a dataframe
for a,b in dirty_data.iterrows():
    
    actual = b['correct_dis_km'] #saving each data value of correct_dis_km to actual
    given = round(b['distance_to_customer_KM'],3) #saving rounding value upto 3 decimals of each data value of 
                                                  #'distance_to_customer_KM' to given
    
    #if given is not equal to actual then fixing it by assigning the correct value.
    if given != actual:
        dirty_data.iloc[a,10]= dirty_data.iloc[a,14] #assigning the correct value to the places where there is an error

        

### Menu check
We need to check if people have ordered from the right menu or not. There may be some error in order_items. There are some rows with order_items from different time menu. For that, we need to calculate the correct menu from outlier data file first. We will create three lists for menu of each time, i.e. breakfast, lunch, dinner.

In [32]:
#first we will create three empty list for storing menu items in each list for breakfast, lunch and dinner.
breakfast=[]
lunch=[]
dinner=[]
for a,b in outlier_data.iterrows():        #performing iteration on all rows of the data
    food = b['order_items']                #storing all the ordered itmes (string type) in variable named 'food'
    o_type = b['order_type']               #storing the type of order (if food is breakfast, lunch or dinner)in o_type variable
    
    if o_type == 'Breakfast':              #if food type is breakfast then store the order items in list named breakfast   
        breakfast.append(food)
    
    if o_type == 'Lunch':                  #if food type is lunch then store the order items in list named lunch
        lunch.append(food)
    
    if o_type == 'Dinner':                 #if food type is dinner then store the order items in list named dinner
        dinner.append(food) 
        
reg_order= re.compile("\('(.*?)',")        #defining a regex to extract the item names from the string of items and quantity.

#coverted the list of all items of dinner into string to perform regex and 
#extracting all the matches for items using regex defined above
dinner_menu=re.findall(reg_order,str(dinner))
dinner_menu=list(set(dinner_menu))         #convert list or items extracted above to set to get unique list of items
print(dinner_menu)

#coverted the list of all items of breakfast into string to perform regex and 
#extracting all the matches for items using regex defined above
breakfast_menu=re.findall(reg_order,str(breakfast))   
breakfast_menu=list(set(breakfast_menu))    #convert list or items extracted above to set to get unique list of items
print(breakfast_menu)

#coverted the list of all items of lunch into string to perform regex and 
#extracting all the matches for items using regex defined above
lunch_menu=re.findall(reg_order,str(lunch))
lunch_menu=list(set(lunch_menu))      #convert list or items extracted above to set to get unique list of items
print(lunch_menu)




['Fish&Chips', 'Pasta', 'Salmon', 'Shrimp']
['Pancake', 'Cereal', 'Coffee', 'Eggs']
['Steak', 'Salad', 'Chicken', 'Burger', 'Fries']


In [33]:
dirty_data.head(7) # row 1 and 6 has order_type lunch and dinner respectively  while its order_item have eggs in it which is an item of breakfast menu

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,customer_node,branch_node,correct_dis_km
0,ORDJ07617,2018-08-17,15:36:20,Lunch,TP,"[('Chicken', 6), ('Steak', 5), ('Burger', 9), ...",730.40,-37.818790,144.955280,0,8.299,11.938508,2247180690,1390575046,8.299
1,ORDY05853,2018-06-26,13:44:47,Lunch,TP,"[('Eggs', 8), ('Chicken', 9), ('Burger', 4), (...",656.20,-37.824714,144.984714,0,8.921,12.535316,746912608,1390575046,8.921
2,ORDI02968,2018-01-26,08:50:42,Breakfast,NS,"[('Coffee', 4), ('Pancake', 7), ('Eggs', 4)]",287.75,-37.824977,144.988823,0,9.692,14.390854,266740437,2455254505,9.692
3,ORDY06446,2018-06-24,13:44:47,Lunch,TP,"[('Steak', 2), ('Chicken', 9), ('Salad', 10)]",550.00,-37.818858,144.974113,0,8.197,13.361016,579402454,1390575046,8.197
4,ORDZ01045,2018-10-02,17:07:36,Dinner,NS,"[('Pasta', 1), ('Fish&Chips', 9), ('Salmon', 8...",778.50,-37.812362,144.973720,0,7.576,13.731754,589398474,2455254505,7.576
5,ORDA06887,2018-03-11,10:11:49,Breakfast,BK,"[('Pancake', 3), ('Eggs', 10), ('Coffee', 2), ...",412.75,-37.803644,144.963391,0,7.674,14.703081,1492410194,1889485053,7.674
6,ORDB08630,2018-11-26,18:49:00,Dinner,TP,"[('Fish&Chips', 9), ('Eggs', 1), ('Shrimp', 7)...",954.00,-37.806155,144.958304,0,9.074,12.801646,777768981,1390575046,9.074


In [34]:
for n,j in dirty_data.iterrows():       #iteration on rows of dataframe 
    order_t=j['order_type']             #storing the order type in variable named order_t to check if the food is breakfast, lunch or dinner
    menu=j['order_items']               #storing the string of items ordered in menu variable
    if order_t=='Breakfast':            #check if order type is breakfast then compare the order items from breakfast menu
        time_menu=breakfast_menu
    if order_t=='Lunch':                #check if order type is lunch then compare the order items from lunch menu
        time_menu=lunch_menu
    if order_t=='Dinner':               #check if order type is dinner then compare the order items from dinner menu
        time_menu=dinner_menu
    
#we need to conert the string of order items into a list of items to compare from the menue and the remove the items which are not in the menue
#for this we firts striped the string and then replaces some parts with '|' to use this as delimeter to split the string into list
    a=menu.strip('([])').replace('), (','|').split('|')
    b=[]               #defining an empty list to store the items which are in the menue
    for x in a:
        if re.findall("'(.*?)'",x)[0] in time_menu:          #using regex, check if each item extracted is in the menue 
            b.append(x)                                      #if present in the menu, add element to list 'b' 
        dirty_data.iloc[n,5]= '[('+'),('.join(b)+')]'        #imputing the list of elements in the form of string the previous pattern into the dataframe



 From the outlier data we used order_items in order to solve the linear equations.
 
 * 4 order_items in from of equations for breakfast
 * 5 order_items in form of equatiions for lunch
 * 4 order_items in form of equations for dinner
 
 We use np.linalg.solve(a,b) which solves a linear matrix equation
 * a is coefficient matrix
 * b is the matrix with price
 
 It returns the unit prices for each order_type

In [35]:


coff_br=np.array([[10, 8, 5, 9],[5, 4, 6, 3],[6, 4, 9, 3],[4, 5, 8, 7]])
coff_dr=np.array([[6, 5, 6, 5],[5, 9, 9, 2],[9, 2, 10, 7],[7, 2, 7, 9]])
coff_lu=np.array([[6, 7, 4, 10, 4],[4, 2, 5, 7, 7],[10, 9, 1, 2, 8],[6, 10, 4, 7, 9],[6, 7, 8, 5, 9]])

dep_br=np.array([645.0,317.25,364.0,414.0])
dep_dr=np.array([869.0,914.5,1118.0,1009.5])
dep_lu=np.array([769.0,679.4,734.4,970.4,1023.0])


break_pr=np.linalg.solve(coff_br,dep_br)
din_pr=np.linalg.solve(coff_dr,dep_dr)
lun_pr=np.linalg.solve(coff_lu,dep_lu)

print('price for fries, burger, steak, salad ,chicken',lun_pr)
print('price for shrimp, fish&chips, pasta ,Salmon',din_pr)
print('price for pancakes, eggs, coffee , cereall',break_pr)

price for fries, burger, steak, salad ,chicken [12.  31.  45.  17.2 32. ]
price for shrimp, fish&chips, pasta ,Salmon [54.  35.  27.5 41. ]
price for pancakes, eggs, coffee , cereall [24.25 22.    7.5  21.  ]


Now, we have the price of each item of all the three food type menu. we will now use this price rate to calculate the total bill for every customer order. Also, if the customer has loyalty equal to one, the customer will gwt 50% discount on his/her shopping.

In [36]:
#defining three list for each food type to store the list of quantity of each items for every food type induvidually.
br = []
lu = []
dr = []
for n,j in dirty_data.iterrows():            #iteration through rows of dataframe
    price = j['order_price']                 #for every row store the price into the variable named price
    items = j['order_items']                 #for every row store the items ordered by the customer into the variable named items
    ord_t = j['order_type']                  #for every row store the food type into the variable named ord_t to check the right menu for comparison

#now, we will calculate the list of elements contating list of quantity of each item for every food type
    if ord_t=='Lunch':                 
    #if food is Lunch store the quantity of fries, burger, steak, salad and chicken in variable a,b,c,d and e respectively
    #using the regex which will extract the digit followed by the item name
        a = ''.join(re.findall("Fries', (\d+)",items))
        b = ''.join(re.findall("Burger', (\d+)",items))
        c = ''.join(re.findall("Steak', (\d+)",items))
        d = ''.join(re.findall("Salad', (\d+)",items))
        e = ''.join(re.findall("Chicken', (\d+)",items))
        
        #if any item is not ordered, i.e is not present in the order_item string, then assign 0 as the quantity
        if a == '':
            a=0
        if b == '':
            b=0
        if c == '':
            c=0
        if d == '':
            d=0
        if e == '':
            e=0
        #append the list of values into the main list for lunch in the form of integer
        lu.append([int(a),int(b),int(c),int(d),int(e)])
        
    if ord_t=='Breakfast':
    #if food is Breakfast store the quantity of pancakes, eggs, coffee and cereal in variable a,b,c and d respectively
    #using the regex which will extract the digit followed by the item name
        a = ''.join(re.findall("Pancake', (\d+)",items))
        b = ''.join(re.findall("Eggs', (\d+)",items))
        c = ''.join(re.findall("Coffee', (\d+)",items))
        d = ''.join(re.findall("Cereal', (\d+)",items))
        
        #if any item is not ordered, i.e is not present in the order_item string, then assign 0 as the quantity
        if a == '':
            a=0
        if b == '':
            b=0
        if c == '':
            c=0
        if d == '':
            d=0
        #append the list of values into the main list for breakfast in the form of integer
        br.append([int(a),int(b),int(c),int(d)])
        
    if ord_t=='Dinner':
    #if food is dinner store the quantity of shrimp, fish&chips, pasta and Salmon in variable a,b,c and d respectively
    #using the regex which will extract the digit followed by the item name
        a = ''.join(re.findall("Shrimp', (\d+)",items))
        b = ''.join(re.findall("Fish&Chips', (\d+)",items))
        c = ''.join(re.findall("Pasta', (\d+)",items))
        d = ''.join(re.findall("Salmon', (\d+)",items))
        
        #if any item is not ordered, i.e is not present in the order_item string, then assign 0 as the quantity
        if a == '':
            a=0
        if b == '':
            b=0
        if c == '':
            c=0
        if d == '':
            d=0
        #append the list of values into the main list for breakfast in the form of integer
        dr.append([int(a),int(b),int(c),int(d)])




We have the list of quantity of all the the items for every food type which can be used to check the price of ordered food.

In [37]:
l,m,o = 0,0,0
#define three flags named l,m and o
for n,j in dirty_data.iterrows(): 
    #for every row save the order price into price variable and type of order into o_t variable
    price = j['order_price']
    o_t = j['order_type']
    if o_t == 'Breakfast':
    #if the type is breakfast, take the first element of the list created above and perform dot product with the list of actual price of the items to get the total price of the data
        dirty_data.iloc[n,6]=round(np.dot(br[l],break_pr),2)  
        #make an increment in the flag to get to the next value for next occurence of breakfast type food
        l+=1
            
    if o_t == 'Lunch':
    #if the type is lunch, take the first element of the list created above and perform dot product with the list of actual price of the items to get the total price of the data
        dirty_data.iloc[n,6]=round(np.dot(lu[m],lun_pr),2) 
        #make an increment in the flag to get to the next value for next occurence of lunch type food
        m+=1
        
    if o_t == 'Dinner':
    #if the type is dinner, take the first element of the list created above and perform dot product with the list of actual price of the items to get the total price of the data
        dirty_data.iloc[n,6]=round(np.dot(dr[o],din_pr),2)
        #make an increment in the flag to get to the next value for next occurence of dinner type food
        o+=1

### Fixing the errors with loyalty 

The approach to fix the loayalty will be to check the delivery fee. The delivery fee depends linearly on 3 variables, time of the day, day of the week and the distance from the customer. We will build a linear model to predict the delivery fee.

Since the missing_data_mod has no errors so we can build(train and test) our model on it and then predict the values for delivery fee on the dirty_data and then check the loyalty according to it.

Before starting we need to add 2 new columns in both the files, dirty_data and missing_data_mod. 

* Column 'time_of_day' tells us the time when the food was ordered (morning 0, afternoon 1, evening 2)
* Column 'weekend_weekday' tells us whether the food was oredered on weekend or weekday (1 or 0)

In [38]:
#adding a new column to dirty_data with all values assigned to 0.
dirty_data['time_of_day']=[0]*dirty_data.shape[0]


#iterrows() function will loop through each row of a dataframe as (index, series) pairs
#for loop along with iterrows will help to iterate thrrough each row one by one.

for a,b in dirty_data.iterrows():
    meal=b['order_type'] #saving each order_type in meal
    
    
    #if meal is equal to Breakfast then assinging value 0 to the column 'time_of_day'
    if meal=='Breakfast':
        dirty_data.iloc[a,15]=0 #iloc[a,15] is datavalue with a as the index of the row and 15 means the column 'time_of_day'
    
    
    #if meal is equal to Lunch then assinging value 1 to the column 'time_of_day'   
    if meal=='Lunch':
        dirty_data.iloc[a,15]=1
        
        
    #if meal is equal to Dinner then assinging value 2 to the column 'time_of_day'    
    if meal =='Dinner':
        dirty_data.iloc[a,15]=2

In [39]:
#adding a new column to dirty_data with all values assigned to 0
dirty_data['weekend_weekday']=[0]*dirty_data.shape[0]

#iterrows() function will loop through each row of a dataframe as (index, series) pairs
#for loop along with iterrows will help to iterate thrrough each row one by one.

for a,b in dirty_data.iterrows():
    date=b['date'] #saving each date in meal
    
    # Timestamp is the pandas equivalent to python’s Datetime
    # It’s used for converting the entries into dates that make up a DatetimeIndex in pandas.
    # df.dayofweek returns the day of week. Monday is treated as 0 and Sunday is treated as 6.
    df = pd.Timestamp(date)
    day=df.dayofweek
    
    #if it is a weekday( 0,1,2,3,4) then assigning the value 0 to the column 'weekend_weekday'
    if (day==0) or (day==1) or (day==1) or (day==2) or (day==3) or (day==4):
        dirty_data.iloc[a,16]=0
        
    #if it is a weekend(5,6) then assigning the value 1 to the column 'weekend_weekday'    
    if (day==5) or (day==6):
        dirty_data.iloc[a,16]=1

Now we load the Group154_missing_data once again into missing_data_mod using pandas

In [40]:
missing_data_mod=pd.read_csv("Group154_missing_data.csv")

In [41]:
missing_data_mod.info()  # .info() gives an insight about which features are categorical and which are numerical.
                         # but we can also see that about 200 values are missing 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
order_id                   500 non-null object
date                       500 non-null object
time                       500 non-null object
order_type                 500 non-null object
branch_code                400 non-null object
order_items                500 non-null object
order_price                500 non-null float64
customer_lat               500 non-null float64
customer_lon               500 non-null float64
customerHasloyalty?        500 non-null int64
distance_to_customer_KM    450 non-null float64
delivery_fee               450 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


Adding similar type of columns to the missing_data_mod

In [42]:
#adding a new column to missing_data_mod with all values assigned to 0.
missing_data_mod['time_of_day']=[0]*missing_data_mod.shape[0]

for a,b in missing_data_mod.iterrows():
    meal=b['order_type']
    
    #if meal is equal to Breakfast then assinging value 0 to the column 'time_of_day'
    if meal=='Breakfast':
        missing_data_mod.iloc[a,12]=0

    #if meal is equal to Lunch then assinging value 1 to the column 'time_of_day'
    if meal=='Lunch':
        missing_data_mod.iloc[a,12]=1
        
    #if meal is equal to Dinner then assinging value 2 to the column 'time_of_day'
    if meal =='Dinner':
        missing_data_mod.iloc[a,12]=2

In [43]:
missing_data_mod['weekend_weekday']=[0]*missing_data_mod.shape[0]

for a,b in missing_data_mod.iterrows():
    time=b['date']
    
    df = pd.Timestamp(time)
    day=df.dayofweek
    
    #if it is a weekday( 0,1,2,3,4) then assigning the value 0 to the column 'weekend_weekday'
    if (day==0) or (day==1) or (day==1) or (day==2) or (day==3) or (day==4):
        missing_data_mod.iloc[a,13]=0
        
    #if it is a weekend(5,6) then assigning the value 1 to the column 'weekend_weekday
    if (day==5) or (day==6):
        missing_data_mod.iloc[a,13]=1

We drop the rows which have missing values so that we bulid our model on the correct information

In [44]:
#.dropna removes the rows with missing values 
# axis =0 means rows 
# how ='any'  means if at least 1 null value is present in the row, drop it.
#inplace=true means do the opeartion inplace

missing_data_mod.dropna( axis=0, how='any', inplace=True)

In next step , we create 3 dataframes w.r.t the branch_code by filtering the missing_data_mod. We need to do this because delievry fee also depends upon the branch and therefore we need to create 3 different models for each branch.

In [45]:
ns=missing_data_mod[missing_data_mod['branch_code']=='NS']
tp=missing_data_mod[missing_data_mod['branch_code']=='TP']
bk=missing_data_mod[missing_data_mod['branch_code']=='BK']

We now only take the 4 columns which are required to train and test

In [46]:
ns = ns[['distance_to_customer_KM','time_of_day','weekend_weekday','delivery_fee']]
tp = tp[['distance_to_customer_KM','time_of_day','weekend_weekday','delivery_fee']]
bk = bk[['distance_to_customer_KM','time_of_day','weekend_weekday','delivery_fee']]

In [47]:
ns.head(2)

,distance_to_customer_KM,time_of_day,weekend_weekday,delivery_fee
7,9.434,1,0,14.808281
14,10.030,0,0,15.022134


##  Model for NS

In [48]:
#Train_test_spliy split arrays or matrices into random train and test subsets with 80% for training and 20 % for testing
#map will convert  into uniform format

X_train_ns, X_test_ns, y_train_ns, y_test_ns = train_test_split(ns.iloc[:,:-1],ns.iloc[:,-1:], random_state = 3,train_size=0.8)
map(pd.np.shape,[X_train_ns, X_test_ns, y_train_ns, y_test_ns])

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [49]:
# we try to fit the linear model 
#include the columns other than delivery fee for X_train and take column'delivery_fee' for y_train
linear_mod_ns = LinearRegression()  #instatiate 
lm_ns = linear_mod_ns.fit(X_train_ns[[x for x in X_train_ns.columns if x!= 'delivery_fee']],y_train_ns['delivery_fee'])

R-squared is a statistical measure of how close the data are to the fitted regression line. It is also known as the coefficient of determination, or the coefficient of multiple determination for multiple regression.

The definition of R-squared is fairly straight-forward; it is the percentage of the response variable variation that is explained by a linear model. Or:

R-squared = Explained variation / Total variation

R-squared is always between 0 and 100%:

0% indicates that the model explains none of the variability of the response data around its mean. 100% indicates that the model explains all the variability of the response data around its mean.

In [50]:
lm_ns.score(X_test_ns,y_test_ns) # calculating the r squard value on X_test and y_test
# 0.9087  means the model is good enough to predict the values

0.9087388527947259

## Model for TP

In [51]:
#Train_test_spliy split arrays or matrices into random train and test subsets with 86% for training and 14 % for testing
#map will convert  into uniform format

X_train_tp, X_test_tp, y_train_tp, y_test_tp = train_test_split(tp.iloc[:,:-1],tp.iloc[:,-1:], random_state=0,train_size=0.86)
map(pd.np.shape,[X_train_tp, X_test_tp, y_train_tp, y_test_tp])

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [52]:
# we try to fit the linear model 
#include the columns other than delivery fee for X_train and take column'delivery_fee' for y_train

linear_mod_tp=LinearRegression() #instatiate 
lm_tp = linear_mod_tp.fit(X_train_tp[[x for x in X_train_tp.columns if x!= 'delivery_fee']],y_train_tp['delivery_fee'])

In [53]:
lm_tp.score(X_test_tp,y_test_tp)# calculating the r squard value on X_test and y_test
# 0.69  means the model is fair enough to predict the values

0.6941741890107581

## Model for BK

In [54]:
#Train_test_spliy split arrays or matrices into random train and test subsets with 83% for training and 17 % for testing.
#map will convert  into uniform format

X_train_bk, X_test_bk, y_train_bk, y_test_bk = train_test_split(bk.iloc[:,:-1],bk.iloc[:,-1:],random_state=0,train_size=0.83)
map(pd.np.shape,[X_train_bk, X_test_bk, y_train_bk, y_test_bk])

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [55]:
# we try to fit the linear model 
#include the columns other than delivery fee for X_train and take column'delivery_fee' for y_train

linear_mod_bk=LinearRegression() #instatiate 
lm_bk=linear_mod_bk.fit(X_train_bk[[x for x in X_train_bk.columns if x!= 'delivery_fee']],y_train_bk['delivery_fee'])

In [56]:
linear_mod_bk.score(X_test_bk,y_test_bk) # calculating the r squard value on X_test and y_test
# 0.836 means the model is good enough to predict the values

0.8360670135955898

We have created all the 3 models and now we will try to predict the values for delivery fee in the dirty_data.
After predicting we will calculate the difference between the predicted value and the actual value. If the differnce is in the range(-1.5,1.5) , then we will assign the loyalty to 0(no 50% off on the delivery fee) since prediction can never be 100% true so nearby values should work. If the difference is out of range(-1.5,1.5) then we assign the loyalty to 1 (50% discount on the delivery fee)

In [57]:
#iterrows() function will loop through each row of a dataframe as (index, series) pairs
#for loop along with iterrows will help to iterate thrrough each row one by one.


for x,y in dirty_data.iterrows():
    b_code=y['branch_code']  #saving each value of different columns to respective varaibles
    d=y['distance_to_customer_KM']
    ti=y['time_of_day']
    w=y['weekend_weekday']
    
    
    #if branch is NS then we use.predict() and pass the array(distance, time of day, weeken_weekday)
    # we calculate and store the difference of actual and the predicted value of delivery fee
    # assign the value to customerHasloyalty? according to the 2 conditions explained above.
    if b_code=='NS':
        
        predict_ns=lm_ns.predict(np.array([[d,ti,w]]))
        difference_ns=dirty_data.iloc[x,11] - predict_ns
        
        if difference_ns < -1.5 or difference_ns >1.5:
            dirty_data.iloc[x,9]=1
        else:
            dirty_data.iloc[x,9]=0
    
    
    #if branch is TP then we use.predict() and pass the array(distance, time of day, weeken_weekday)
    # we calculate and store the difference of actual and the predicted value of delivery fee
    # assign the value to customerHasloyalty? according to the 2 conditions explained above.
    if b_code=='TP':
        
        predict_tp=lm_tp.predict(np.array([[d,ti,w]]))
        difference_tp=dirty_data.iloc[x,11] - predict_tp
        
        if difference_tp < -1.5 or difference_tp >1.5:
            dirty_data.iloc[x,9]=1
        else:
            dirty_data.iloc[x,9]=0
            
    #if branch is BK then we use.predict() and pass the array(distance, time of day, weeken_weekday)
    # we calculate and store the difference of actual and the predicted value of delivery fee
    # assign the value to customerHasloyalty? according to the 2 conditions explained above.       
    if b_code=='BK':
        
        predict_bk=lm_bk.predict(np.array([[d,ti,w]]))
        difference_bk=dirty_data.iloc[x,11] - predict_bk
        
        if difference_bk < -1.5 or difference_bk >1.5:
            dirty_data.iloc[x,9]=1
        else:
            dirty_data.iloc[x,9]=0


All the errors have been fixed, now we drop the extra columns we added and save the dataframe as csv file.

In [58]:
dirty_data=dirty_data.drop(['customer_node','branch_node','correct_dis_km','time_of_day','weekend_weekday'],axis=1)

In [59]:
dirty_data.to_csv('Group154_dirty_data_solution.csv')

## Detecting and removing outlier rows in outlier_data

The outliers are found w.r.t delivery fee. Since delivery fee is dependent variable so it is wise to find out the outliers of the residual (difference between the actual and predicted value of delievry fee). We will use the same 3 models(branch_wise) we created on the missing data and then predict the delivery fee values. After predicting the values we will calculate the difference and store it into the new column 'residual' we create. We will find the outliers branch wise on the residual columns and then remove them.

We add 2 columns in a similar way we added in the dirty_data

In [60]:
#adding a new column to outlier_data with all values assigned to 0.
outlier_data['time_of_day']=[0]*outlier_data.shape[0]

for a,b in outlier_data.iterrows():
    meal=b['order_type']
    
    #if meal is equal to Breakfast then assinging value 0 to the column 'time_of_day'
    if meal=='Breakfast':
        outlier_data.iloc[a,12]=0
    
    #if meal is equal to Lunch then assinging value 1 to the column 'time_of_day'
    if meal=='Lunch':
        outlier_data.iloc[a,12]=1
        
    #if meal is equal to Dinner then assinging value 2 to the column 'time_of_day'    
    if meal =='Dinner':
        outlier_data.iloc[a,12]=2
        


In [61]:
#adding a new column to outlier_data with all values assigned to 0.
outlier_data['weekend_weekday']=[0]*outlier_data.shape[0]

for a,b in outlier_data.iterrows():
    time=b['date']
    
    df = pd.Timestamp(time)
    day=df.dayofweek
    

    #if it is a weekday( 0,1,2,3,4) then assigning the value 0 to the column 'weekend_weekday'
    if (day==0) or (day==1) or (day==1) or (day==2) or (day==3) or (day==4):
        outlier_data.iloc[a,13]=0
        
    #if it is a weekend(5,6) then assigning the value 1 to the column 'weekend_weekday
    if (day==5) or (day==6):
        outlier_data.iloc[a,13]=1

We add another column 'residual' and assigning all the values to be 0 

In [62]:
outlier_data['residual']=[0]*outlier_data.shape[0]

We will use the same 3 models and we predict the values for delivery fee in the outlier_data. After predicting we will first check whether the customer has loyalty or not. IF loyalty is 0 then we will calculate the difference between the predicted value and the actual value and if loyalty is 1 , then we will calculate the difference between the (predicted value/2) and the actual value since loyalty being 1 means customer has 50% off on the delievry fee.

In [63]:
for x,y in outlier_data.iterrows():
    #performing iteration on every row of the data to check and filter out the outliers of the delivery fee
    loyal = y['customerHasloyalty?']
    #customer loyalty is stored in variable named 'loyal' used to predict the delivery charge and provide discount 
    b_code=y['branch_code']
    #branch code is saved in 'b_code' variable used to use the respective model for prediction
    d=y['distance_to_customer_KM']
    #distance is stored to calculate the delivery fee
    ti=y['time_of_day']
    w=y['weekend_weekday']
    #time of day and weekday used in prediction model is stored in variable ti and w
    
    if b_code=='NS':
        #residual (difference between actual and predicted delivery fee) for branch with branch code 'NS'
        pred_ns=lm_ns.predict(np.array([[d,ti,w]]))
        if loyal==0:
            #if customer loyalty is zero, no discount applied and delivery fee will be as predicted
            residual_ns = outlier_data.iloc[x,11] - pred_ns
            #residual will be calculated using the predicted value
        else:
            #otherwise, customers with loyalty card will get 50% dicsount on delivery fee
            residual_ns = outlier_data.iloc[x,11] - (pred_ns/2)
            #residual will be calculated using the half of the predicted value
        outlier_data.iloc[x,14] = residual_ns
        #storing the residual value to the residual column added to the dataframe
    
    if b_code=='TP':
        #residual (difference between actual and predicted delivery fee) for branch with branch code 'TP'
        pred_tp=lm_tp.predict(np.array([[d,ti,w]]))
        if loyal==0:
            #if customer loyalty is zero, no discount applied and delivery fee will be as predicted
            residual_tp=outlier_data.iloc[x,11] - pred_tp
            #residual will be calculated using the predicted value
        else:
            #otherwise, customers with loyalty card will get 50% dicsount on delivery fee
            residual_tp=outlier_data.iloc[x,11] - (pred_tp/2)
            #residual will be calculated using the half of the predicted value
        outlier_data.iloc[x,14] = residual_tp
        #storing the residual value to the residual column added to the dataframe
        
            
    if b_code=='BK':
        #residual (difference between actual and predicted delivery fee) for branch with branch code 'BK'
        pred_bk=lm_bk.predict(np.array([[d,ti,w]]))
        if loyal==0:
            #if customer loyalty is zero, no discount applied and delivery fee will be as predicted
            residual_bk=outlier_data.iloc[x,11] - pred_bk
            #residual will be calculated using the predicted value
        else:
            #otherwise, customers with loyalty card will get 50% dicsount on delivery fee
            residual_bk=outlier_data.iloc[x,11] - (pred_bk/2)
            #residual will be calculated using the half of the predicted value
        outlier_data.iloc[x,14] = residual_bk
        #storing the residual value to the residual column added to the dataframe




In [64]:
outlier_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,time_of_day,weekend_weekday,residual
0,ORDC03038,2018-04-03,11:02:32,Breakfast,NS,"[('Eggs', 5), ('Pancake', 10)]",352.50,-37.812274,144.989074,0,8.737,13.376205,0,0,0.361546
1,ORDA07743,2018-05-30,12:54:05,Lunch,BK,"[('Steak', 4), ('Chicken', 4), ('Burger', 7), ...",769.00,-37.804380,144.929732,0,10.562,16.460243,1,0,0.673049
2,ORDI05524,2018-02-12,09:21:07,Breakfast,NS,"[('Cereal', 8), ('Eggs', 8), ('Coffee', 7)]",396.50,-37.807646,144.954601,0,8.248,13.185125,0,0,0.635502
3,ORDI06303,2018-08-12,15:26:11,Lunch,NS,"[('Fries', 4), ('Chicken', 7), ('Burger', 2), ...",679.40,-37.813534,144.969359,0,7.684,14.961276,1,1,0.004752
4,ORDZ08782,2018-04-16,11:22:49,Breakfast,NS,"[('Cereal', 9), ('Coffee', 5), ('Eggs', 8), ('...",645.00,-37.810355,144.969275,0,7.237,12.269602,0,0,0.681435
5,ORDA06891,2018-01-19,08:30:25,Breakfast,BK,"[('Cereal', 10), ('Eggs', 1), ('Pancake', 7)]",401.75,-37.823537,144.980873,0,6.984,12.103057,0,0,1.103528
6,ORDY01417,2018-01-13,08:20:16,Breakfast,TP,"[('Pancake', 9), ('Coffee', 1), ('Eggs', 6)]",357.75,-37.819273,144.988521,1,9.472,7.300420,0,1,1.176888
7,ORDA07013,2018-08-19,15:36:20,Lunch,BK,"[('Salad', 3), ('Steak', 4), ('Chicken', 4), (...",607.60,-37.804300,144.945835,0,9.060,17.580106,1,1,0.741182
8,ORDJ04617,2018-12-15,19:29:34,Dinner,TP,"[('Fish&Chips', 3), ('Shrimp', 9)]",591.00,-37.824387,144.943728,0,9.150,14.915326,2,1,0.241447
9,ORDY04176,2018-09-09,16:16:54,Dinner,TP,"[('Fish&Chips', 5), ('Shrimp', 6), ('Salmon', ...",869.00,-37.810193,144.961383,0,8.501,14.338527,2,1,0.145074


We need to show the boxplot for the delivery fee to get the idea about the outliers.

For filtering the outliers we will perform plot on the residual, i.e the difference between the actual value and the predicted one using the model



In [65]:
outlier_data.boxplot(by='branch_code', column = 'residual') 
#boxplot on residual(difference between actual and predicted value) group by branch

The above boxplot shows the median, 1st,2nd and 3rd quartile range for each branch and the values of the outliers for all three branches induvidually. Branch with branchcode BK has only 4 outlier values, whereas branch with branch code TP has the maximum number of outlier values.

Now, we can see from the plot above that there are some considerable number of outliers in the data for each branch, which needs to be filtered and removed from the data. Wr need to calculate the Upper bounf and Lower bound for the residual to calculate the filter the outliers. This can be calculated using formula:
$$UB = Q_3 + 1.5\times IQR$$


$$LB = Q_1 - 1.5\times IQR$$


$$IQR = Q_3 - Q_1$$
where, $Q_1$ is first quartile and $Q_4$ is fourth quartile and $IQR$ is the inter quartile range
Outliers are the unrealistic values that are not relatable to the data. These values are the values above the upper bound and values less than the lower bound.

In [66]:
#calculating the quartiles for residual of branch with branch code 'NS'
vis_ns=outlier_data[outlier_data['branch_code']=='NS'].residual.describe()
#calculating IQR using Q3 and Q1 and using IQR to calculate the upper and lower bound using the formula above
iqr_ns = vis_ns[6]-vis_ns[4]
ub_ns = vis_ns[6] + 1.5*iqr_ns
lb_ns = vis_ns[4] - 1.5*iqr_ns

#calculating the quartiles for residual of branch with branch code 'TP'
vis_tp=outlier_data[outlier_data['branch_code']=='TP'].residual.describe()
#calculating IQR using Q3 and Q1 and using IQR to calculate the upper and lower bound using the formula above
iqr_tp = vis_tp[6]-vis_tp[4]
ub_tp = vis_tp[6] + 1.5*iqr_tp
lb_tp = vis_tp[4] - 1.5*iqr_tp

#calculating the quartiles for residual of branch with branch code 'BK'
vis_bk=outlier_data[outlier_data['branch_code']=='BK'].residual.describe()
#calculating IQR using Q3 and Q1 and using IQR to calculate the upper and lower bound using the formula above
iqr_bk = vis_bk[6]-vis_bk[4]
ub_bk = vis_bk[6] + 1.5*iqr_bk
lb_bk = vis_bk[4] - 1.5*iqr_bk



Now, as we have calculated the upper bound and lower bound values of residual for each branch type we can filter the rows with outlier.

In [67]:
#storing the indexes for rows with redisual outliers with value more than upper bound and less than lower bound for branch 'NS'
o_ns = outlier_data[(outlier_data['branch_code']=='NS') & ((outlier_data['residual'] > ub_ns) | (outlier_data['residual'] < lb_ns))].index.values.astype(int)

#storing the indexes for rows with redisual outliers with value more than upper bound and less than lower bound for branch 'TP'
o_tp = outlier_data[(outlier_data['branch_code']=='TP') & ((outlier_data['residual'] > ub_tp) | (outlier_data['residual'] < lb_tp))].index.values.astype(int)

#storing the indexes for rows with redisual outliers with value more than upper bound and less than lower bound for branch 'BK'
o_bk = outlier_data[(outlier_data['branch_code']=='BK') & ((outlier_data['residual'] > ub_bk) | (outlier_data['residual'] < lb_bk))].index.values.astype(int)

#the above variables carry the indexes in the form of a one-d array

In [68]:
#first we will convert the array into list and then perform concatination to get a final list of rows with outliers
index = list(o_ns) + list(o_tp) + list(o_bk)
len(index)
#in total there are 20 outtlier values which needs to be removed, 

#droping all the rows with index in list using drop function
outlier_data = outlier_data.drop(index, axis = 0)

In [69]:
#after droping 20 rows with outliers, we left with 480
outlier_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 499
Data columns (total 15 columns):
order_id                   480 non-null object
date                       480 non-null object
time                       480 non-null object
order_type                 480 non-null object
branch_code                480 non-null object
order_items                480 non-null object
order_price                480 non-null float64
customer_lat               480 non-null float64
customer_lon               480 non-null float64
customerHasloyalty?        480 non-null int64
distance_to_customer_KM    480 non-null float64
delivery_fee               480 non-null float64
time_of_day                480 non-null int64
weekend_weekday            480 non-null int64
residual                   480 non-null float64
dtypes: float64(6), int64(3), object(6)
memory usage: 60.0+ KB


The outliers have been removed , now we drop the extra columns we added and later save it into csv file

In [70]:
outlier_data=outlier_data.drop(['time_of_day','weekend_weekday','residual'],axis=1)

In [71]:
outlier_data.to_csv('Group154_outlier_data_solution.csv')

## Imputing the missing values in the missing_data.

In [72]:
missing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
order_id                   500 non-null object
date                       500 non-null object
time                       500 non-null object
order_type                 500 non-null object
branch_code                400 non-null object
order_items                500 non-null object
order_price                500 non-null float64
customer_lat               500 non-null float64
customer_lon               500 non-null float64
customerHasloyalty?        500 non-null int64
distance_to_customer_KM    450 non-null float64
delivery_fee               450 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


We observe that there are:
* 100 values missing in the branch_code
* 50 values missing in the customerHasloyalty?
* 50 values missing in the delivery_fee

<b>We first impute the missing_values in branch_code</b>. We know that branch_code and order_id have some relation. From the outlier_data we extracted the 3 lists corresponding to different branches. We will do the same sort of thing.

In [73]:
#iterrows() function will loop through each row of a dataframe as (index, series) pairs
#for loop along with iterrows will help to iterate thrrough each row one by one.

for n,j in missing_data.iterrows():
    if pd.isnull(j['branch_code']): # .isnull() will check wherever the branch_code is null
        
        #if first 4 charcters of order_id are in the list
        # then assign 'NS' to the corresponding branch_code
        if j['order_id'][0:4] in ['ORDZ', 'ORDC', 'ORDI']:
            missing_data.iloc[n,4]='NS'
            
        #if first 4 charcters of order_id are in the list
        # then assign 'TP' to the corresponding branch_code      
        if j['order_id'][0:4] in ['ORDY', 'ORDJ', 'ORDB']:
            missing_data.iloc[n,4]='TP'
            
        #if first 4 charcters of order_id are in the list
        # then assign 'BK' to the corresponding branch_code
        if j['order_id'][0:4] in ['ORDK', 'ORDX', 'ORDA']:
            missing_data.iloc[n,4]='BK'


<b>Imputing the missing values is distance_to_customer_KM
    
    

We first need to merge the dataframes and we do it by using merge. "pd.merge" merges two DataFrames or named Series objects with a database-style join. We first perform a left join on missing_data and nodes_data so that to corresponding customer lat and lon we get the correct customer nodes.

In [74]:
missing_data = pd.merge(left=missing_data,right=nodes_data, how='left', left_on=['customer_lat','customer_lon'], right_on=['lat','lon'])
#we are merging on the same dataframe

missing_data=missing_data.drop(columns=['lat','lon'])
missing_data=missing_data.rename(columns={'node':'customer_node'})

### There is an error in the nodes.csv file as get 502 rows after joining(left join) the two dfs.

In [75]:
missing_data.count()

order_id                   502
date                       502
time                       502
order_type                 502
branch_code                502
order_items                502
order_price                502
customer_lat               502
customer_lon               502
customerHasloyalty?        502
distance_to_customer_KM    452
delivery_fee               452
customer_node              502
dtype: int64

### We try to find out where the errors are so that we do not get further problems. We find out that there are 2 extra rows for the same order_id

In [76]:
missing_data.order_id.value_counts().head(3)

ORDJ06473    3
ORDC01603    1
ORDA03332    1
Name: order_id, dtype: int64

### We look into this order_id

In [77]:
missing_data[missing_data['order_id']=='ORDJ06473']

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,customer_node
394,ORDJ06473,2018-09-25,16:47:19,Dinner,TP,"[('Salmon', 3), ('Fish&Chips', 8), ('Shrimp', ...",811.5,-37.819726,144.969655,0,8.164,12.49497,243963290
395,ORDJ06473,2018-09-25,16:47:19,Dinner,TP,"[('Salmon', 3), ('Fish&Chips', 8), ('Shrimp', ...",811.5,-37.819726,144.969655,0,8.164,12.49497,6182893637
396,ORDJ06473,2018-09-25,16:47:19,Dinner,TP,"[('Salmon', 3), ('Fish&Chips', 8), ('Shrimp', ...",811.5,-37.819726,144.969655,0,8.164,12.49497,6182893638


### It is strange to see such kind of scenario where for 3 different nodes we have same latitude and longitude.

In [78]:
nodes_data[(nodes_data['node']== 243963290) | (nodes_data['node']== 6182893637) | (nodes_data['node']== 6182893638)]

,node,lat,lon
5320,243963290,-37.819726,144.969655
12603,6182893637,-37.819726,144.969655
12604,6182893638,-37.819726,144.969655


### We drop the two rows 395 and 396 from our outlier_data 

In [79]:
missing_data = missing_data.drop([395,396],axis=0)

In [80]:
missing_data.count()

order_id                   500
date                       500
time                       500
order_type                 500
branch_code                500
order_items                500
order_price                500
customer_lat               500
customer_lon               500
customerHasloyalty?        500
distance_to_customer_KM    450
delivery_fee               450
customer_node              500
dtype: int64

#### Now it looks better
Since we already have a datframe branches 1 which have its code and node. We now need to perform left join on branches1 and outlier_data so that to to corresponding branch code we get correct branch node.

In [81]:
missing_data=pd.merge(left=missing_data,right=branches1, how='left', left_on=['branch_code'], right_on=['branch_code'])

<b>Now can apply Djisktra algorithim

In [82]:
source = missing_data['branch_node']  #storing each 'branch_code' in source from the dirty_data
target = missing_data['customer_node']  #storing each 'customer_code' in target from the dirty_data

temp_list2=list(zip(source,target)) #zipping the variables and storing them in list
dist1=[] #empty list

for a,b in temp_list2:
    dist1.append(networkx.dijkstra_path_length(G,a,b))
    #dijkstra_path_length returns the shortest path length from source to target in a weighted graph.
    #appending all the distances to the list dist1

In [83]:
#The distance we calculated is in metres and we need to convert it into km since the distance_to_customer_KM is given in kms.
#we divide each item of the list by 1000 using for loop and appending the new items into new list 'dis_out'

dis_out=[] #empty list
for x in dist1:
    dis_out.append(x/1000)
    

In [84]:
#iterrows() function will loop through each row of a dataframe as (index, series) pairs
#for loop along with iterrows will help to iterate thrrough each row one by one.

for n,j in missing_data.iterrows():
    if pd.isnull(j['distance_to_customer_KM']): #.isnull() will check wherever the distance_to_customer_KM is null
        missing_data.iloc[n,10]=dis_out[n]      # and we will then assign the distance we calculated at the missing places
                                                # n is the index of the row and index of the dis_out as well

### Imputing the missing values in the delivery fee column

We first need to add 2 columns in similar way we added above to missing_data as well

In [85]:
#adding a new column to missing_data with all values assigned to 0.
missing_data['time_of_day']=[0]*missing_data.shape[0]

for a,b in missing_data.iterrows():
    meal=b['order_type']
    
    #if meal is equal to Breakfast then assinging value 0 to the column 'time_of_day'
    if meal=='Breakfast':
        missing_data.iloc[a,14]=0
        
    #if meal is equal to Lunch then assinging value 1 to the column 'time_of_day'
    if meal=='Lunch':
        missing_data.iloc[a,14]=1
        
    #if meal is equal to Dinner then assinging value 2 to the column 'time_of_day'
    if meal =='Dinner':
        missing_data.iloc[a,14]=2

In [86]:
#adding a new column to missing_data with all values assigned to 0.
missing_data['weekend_weekday']=[0]*missing_data.shape[0]

for a,b in missing_data.iterrows():
    time=b['date']
    
    df = pd.Timestamp(time)
    day=df.dayofweek
    
    
    #if it is a weekday( 0,1,2,3,4) then assigning the value 0 to the column 'weekend_weekday'
    if (day==0) or (day==1) or (day==1) or (day==2) or (day==3) or (day==4):
        missing_data.iloc[a,15]=0
        
    #if it is a weekend(5,6) then assigning the value 1 to the column 'weekend_weekday
    if (day==5) or (day==6):
        missing_data.iloc[a,15]=1

We will use the same 3 models we created from the missing_data_mod and we predict the values for delivery fee in the missing_data. After predicting we will first check whether the customer has loyalty or not. IF loyalty is 0 then we will assign the predicted value and if loyalty is 1 , then we will assign the (predicted value/2) since loyalty being 1 means customer has 50% off on the delievry fee. 

In [87]:
for x,y in missing_data.iterrows():
    #performing iteration on every row of the data to check and fill the missing values of the delivery fee
    lo = y['customerHasloyalty?']
    #customer loyalty is stored in variable named 'lo' used to predict the delivery charge and provide discount 
    code=y['branch_code']
    #branch code is saved in 'code' variable used to use the respective model for prediction
    di=y['distance_to_customer_KM']
    #distance is stored to calculate the delivery fee
    t=y['time_of_day']
    ww=y['weekend_weekday']
    #time of day and weekday used in prediction model is stored in variable t and ww
    
    #if the value of delivery fee is null(missing) we will fill it using our prediction model
    if pd.isnull(j['delivery_fee']):
    
        if code=='NS':
            #for null delivery fee for branch with branch code 'NS'
            p_ns=lm_ns.predict(np.array([[di,t,ww]])) #using model 'lm_ns' tp predict the fee using distance, day time and weekday
            if lo==0:
            #if customer loyalty is zero, no discount applied and delivery fee will be as predicted
                missing_data.iloc[x,11] = p_ns
            else:
                #otherwise, customers with loyalty card will get 50% dicsount on delivery fee
                missing_data.iloc[x,11] = p_ns/2
    
        if code=='TP':
            #for null delivery fee for branch with branch code 'TP'
            p_tp=lm_tp.predict(np.array([[di,t,ww]]))
            if lo==0:
                #if customer loyalty is zero, no discount applied and delivery fee will be as predicted
                missing_data.iloc[x,11] = p_tp
            else:
                #otherwise, customers with loyalty card will get 50% dicsount on delivery fee
                missing_data.iloc[x,11] = p_tp/2
        
        if code=='BK':
            #for null delivery fee for branch with branch code 'BK'
            p_bk=lm_bk.predict(np.array([[di,t,ww]]))
            if lo==0:
                #if customer loyalty is zero, no discount applied and delivery fee will be as predicted
                missing_data.iloc[x,11] = p_bk
            else:
                #otherwise, customers with loyalty card will get 50% dicsount on delivery fee
                missing_data.iloc[x,11] = p_bk/2

All the missing values have been imputed , now we drop the extra columns we added and then we save the dataframe to into csv file

In [92]:
missing_data=missing_data.drop(['time_of_day','weekend_weekday','customer_node','branch_node'],axis=1)

In [93]:
missing_data.to_csv('Group154_outlier_data_solution.csv')

In [94]:
missing_data.head()

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDC06762,2018-07-01,13:54:55,Lunch,NS,"[('Burger', 7), ('Steak', 2), ('Salad', 4)]",375.8,-37.803909,144.957357,0,5.380,12.765431
1,ORDK08784,2018-02-05,09:10:59,Breakfast,BK,"[('Cereal', 10), ('Pancake', 8)]",404.0,-37.814946,144.964766,0,7.655,11.595206
2,ORDZ03927,2018-08-07,15:16:03,Lunch,NS,"[('Steak', 2), ('Fries', 7), ('Chicken', 7)]",398.0,-37.803134,144.961925,0,7.090,12.359811
3,ORDZ06176,2018-12-20,19:39:43,Dinner,NS,"[('Pasta', 2), ('Salmon', 9)]",424.0,-37.812911,144.933698,0,8.905,14.997308
4,ORDJ05346,2018-08-24,15:46:28,Lunch,TP,"[('Salad', 10), ('Fries', 10), ('Burger', 3)]",385.0,-37.803353,144.951851,0,9.815,12.773022
